In [1]:
#Mount my drive- run the code, go to the link, accept.
from google.colab import drive
drive.mount('/content/gdrive')

#Change working directory to make it easier to access the files
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/dinn")
os.getcwd() 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'/content/gdrive/My Drive/Colab Notebooks/dinn'

In [2]:
import torch
from torch.autograd import grad
import torch.nn as nn
from numpy import genfromtxt
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

tSI_vaccination_data = genfromtxt('tSI_vaccination_data.csv', delimiter=',') #in the form of [t, S, I]

torch.manual_seed(1234)

In [3]:
%%time

PATH = 'vacc_tensor' 

class DINN(nn.Module):
    def __init__(self, t, S_data, I_data): #t, S_data, I_data
        super(DINN, self).__init__()
        self.t = torch.tensor(t, requires_grad=True)
        self.t_float = self.t.float()
        self.t_batch = torch.reshape(self.t_float, (len(self.t),1)) #reshape for batch 
        self.S = torch.tensor(S_data) 
        self.I = torch.tensor(I_data) 

        self.losses = [] #keep the losses
        self.save = 3 #which file to save to
 
        #learnable parameters
        self.alpha1_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))
        self.alpha2_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))
        self.beta_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))
        self.mu_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))
        self.u_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))
        self.tao_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))

        #matrices (x2 for S,I) for the gradients
        self.m1 = torch.zeros((len(self.t), 2)); self.m1[:, 0] = 1
        self.m2 = torch.zeros((len(self.t), 2)); self.m2[:, 1] = 1

        #NN
        self.net_si = self.Net_si()
        self.params = list(self.net_si.parameters())
        self.params.extend(list([self.mu_tilda, self.beta_tilda, self.alpha1_tilda, self.alpha2_tilda, self.u_tilda, self.tao_tilda]))

        
    #force parameters to be in a range
    @property
    def alpha1(self):
        return torch.tanh(self.alpha1_tilda) * 0.1 + 1 

    @property
    def alpha2(self):
        return torch.tanh(self.alpha2_tilda) * 0.1

    @property
    def beta(self):
        return torch.tanh(self.beta_tilda) * 0.01

    @property
    def mu(self):
        return torch.tanh(self.mu_tilda) * 6 + 6

    @property
    def u(self):
        return torch.tanh(self.u_tilda) * 0.1 + 0.5

    @property
    def tao(self):
        return torch.tanh(self.tao_tilda) * 0.1 + 0.5

    #nets
    class Net_si(nn.Module): # input = [t]
        def __init__(self):
            super(DINN.Net_si, self).__init__()
            self.fc1=nn.Linear(1, 20) #takes 100 t's
            self.fc2=nn.Linear(20, 20)
            self.fc3=nn.Linear(20, 20)
            self.fc4=nn.Linear(20, 20)
            self.fc5=nn.Linear(20, 20)
            self.fc6=nn.Linear(20, 20)
            self.fc7=nn.Linear(20, 20)
            self.fc8=nn.Linear(20, 20)
            self.out=nn.Linear(20, 2) #outputs S, I

        def forward(self, t):
            si=F.relu(self.fc1(t))
            si=F.relu(self.fc2(si))
            si=F.relu(self.fc3(si))
            si=F.relu(self.fc4(si))
            si=F.relu(self.fc5(si))
            si=F.relu(self.fc6(si))
            si=F.relu(self.fc7(si))
            si=F.relu(self.fc8(si))
            si=self.out(si)
            return si    

    def net_f(self, t_batch):       
        si = self.net_si(t_batch)

        S,I = si[:,0], si[:,1]

        #S_t
        si.backward(self.m1, retain_graph=True)
        S_t = self.t.grad
        self.t.grad.zero_()

        #I_t
        si.backward(self.m2, retain_graph=True)
        I_t = self.t.grad
        self.t.grad.zero_()

        f1 = S_t + self.beta * S * I - self.u * torch.reshape(torch.nn.Sigmoid()(-1e3*(self.tao - t_batch)),(1,100)) * (-self.alpha1) * S 
        f2 = I_t  - self.beta * S * I + self.mu * I - self.u * torch.reshape(torch.nn.Sigmoid()(-1e3*(self.tao - t_batch)),(1,100)) * (-self.alpha2) * I 
        return f1, f2, S, I
    
    def load(self):
      # Load checkpoint
      try:
        print('trying to load...')
        checkpoint = torch.load(PATH + str(self.save)+'.pt') 
        print('\nloading pre-trained model...')
        self.load_state_dict(checkpoint['model'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        self.losses = checkpoint['losses']
        print('loaded previous loss: ', loss)
      except RuntimeError :
          print('changed the architecture, ignore')
          pass
      except FileNotFoundError:
          pass

    def train(self, n_epochs):
      #try loading
      self.load()

      #train
      print('\nstarting training...\n')
      
      for epoch in range(n_epochs):
        #lists to hold the output (maintain only the final epoch)
        S_pred_list= []
        I_pred_list= []


        f1, f2, S_pred, I_pred = self.net_f(self.t_batch)
        self.optimizer.zero_grad()

        S_pred_list.append(S_pred)
        I_pred_list.append(I_pred)

        loss =  (torch.mean(torch.square(self.S - S_pred)) + torch.mean(torch.square(self.I - I_pred))+
                torch.mean(torch.square(f1)) + torch.mean(torch.square(f2)))

        loss.backward()

        self.optimizer.step()
        self.scheduler.step() #scheduler

        self.losses.append(loss)

        #loss + model parameters update
        if epoch % 1000 == 0:
          #checkpoint save every 1000 epochs if the loss is lower
          print('\nSaving model... Loss is: ', loss)
          torch.save({
              'epoch': epoch,
              'model': self.state_dict(),
              'optimizer_state_dict': self.optimizer.state_dict(),
              'scheduler': self.scheduler.state_dict(),
              'loss': loss,
              'losses': self.losses,
              }, PATH + str(self.save)+'.pt')
          if self.save % 2 > 0: #its on 3
            self.save = 2 #change to 2
          else: #its on 2
            self.save = 3 #change to 3

          print('epoch: ', epoch)
          print('alpha1: (goal 1)', self.alpha1)
          print('\nalpha2: (goal 0)', self.alpha2)
          print('\nbeta: (goal 0.0075): ', self.beta)
          print('\nmu (goal 5): ', self.mu)
          print('\nu: (goal 0.515151515): ', self.u)
          print('\ntao (goal 0.58): ', self.tao)
          print('#################################')
        
      #plot
      plt.plot(self.losses, color = 'teal')
      plt.xlabel('Epochs')
      plt.ylabel('Loss')
      return S_pred_list, I_pred_list

CPU times: user 63 µs, sys: 0 ns, total: 63 µs
Wall time: 67.2 µs


In [ ]:
%%time

#this worked best

dinn = DINN(tSI_vaccination_data[0], tSI_vaccination_data[1], tSI_vaccination_data[2]) #t, S_data, I_data

learning_rate = 0.02
optimizer = optim.Adam(dinn.params, lr = learning_rate)
dinn.optimizer = optimizer

scheduler = torch.optim.lr_scheduler.CyclicLR(dinn.optimizer, base_lr=1e-6, max_lr=5e-4, step_size_up=3000, mode="triangular2", cycle_momentum=False)
dinn.scheduler = scheduler

S_pred_list, I_pred_list = dinn.train(100000) #train

trying to load...

loading pre-trained model...
loaded previous loss:  tensor(44636.1911, dtype=torch.float64, requires_grad=True)

starting training...


Saving model... Loss is:  tensor(44636.1912, dtype=torch.float64, grad_fn=<AddBackward0>)
epoch:  0
alpha1: (goal 1) tensor([0.9018], grad_fn=<AddBackward0>)

alpha2: (goal 0) tensor([0.0791], grad_fn=<MulBackward0>)

beta: (goal 0.0075):  tensor([0.0099], grad_fn=<MulBackward0>)

mu (goal 5):  tensor([11.4339], grad_fn=<AddBackward0>)

u: (goal 0.515151515):  tensor([0.4018], grad_fn=<AddBackward0>)

tao (goal 0.58):  tensor([0.5910], grad_fn=<AddBackward0>)
#################################

Saving model... Loss is:  tensor(44636.0564, dtype=torch.float64, grad_fn=<AddBackward0>)
epoch:  1000
alpha1: (goal 1) tensor([0.9018], grad_fn=<AddBackward0>)

alpha2: (goal 0) tensor([0.0791], grad_fn=<MulBackward0>)

beta: (goal 0.0075):  tensor([0.0099], grad_fn=<MulBackward0>)

mu (goal 5):  tensor([11.4340], grad_fn=<AddBackward0>)

u: (

In [ ]:
plt.plot(dinn.losses[90000:], color = 'teal')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
ax.plot(tSI_vaccination_data[0], tSI_vaccination_data[1], 'navy', alpha=0.9, lw=2, label='Susceptible')
ax.plot(tSI_vaccination_data[0], S_pred_list[0].detach().numpy(), 'violet', alpha=0.9, lw=2, label='Susceptible Prediction', linestyle='dashed')
ax.plot(tSI_vaccination_data[0], tSI_vaccination_data[2], 'dodgerblue', alpha=0.9, lw=2, label='Infected')
ax.plot(tSI_vaccination_data[0], I_pred_list[0].detach().numpy(), 'darkgreen', alpha=0.9, lw=2, label='Infected Prediction', linestyle='dashed')

ax.set_xlabel('Time /days')
ax.set_ylabel('Number')
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()

In [ ]:
print('alpha1: (goal 1)', round(dinn.alpha1.item(),2))
print('\nalpha2: (goal 0)', round(dinn.alpha2.item(),2))
print('\nbeta: (goal 0.0075): ', round(dinn.beta.item(),4))
print('\nmu (goal 5): ', round(dinn.mu.item(),2))
print('\nu: (goal 0.515151515): ', round(dinn.u.item(),2))
print('\ntao (goal 0.58): ', round(dinn.tao.item(),2))


print('\nerror:')
print('alpha1: ', round((1-round(dinn.alpha1.item(),2))/1,2)*100,'%')
print('alpha2: ', round((0-round(dinn.alpha2.item(),2))/1e-20,2)*100,'%')
print('beta: ', round((0.0075-round(dinn.beta.item(),4))/0.0075,2)*100,'%')
print('mu: ', round((5-round(dinn.mu.item(),2))/5,2)*100,'%')
print('u: ', round((0.515151515-round(dinn.u.item(),2))/0.515151515,2)*100,'%')
print('tao: ', round((0.58-round(dinn.tao.item(),2))/0.58,2)*100,'%')

In [ ]:
#vaccination! 

import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Initial number of infected individuals, I0
I0 = 1
# Everyone else, S0, is susceptible to infection initially.
S0 = 2000
# Contact rate, beta, and mean recovery rate, mu.
beta, mu = dinn.beta, dinn.mu
# A grid of time points (in days)
t = np.linspace(0, 3, 100) 
#parameters
u = dinn.u
tao = dinn.tao
alpha1 = dinn.alpha1
alpha2 = dinn.alpha2

# The SIR model differential equations.
def deriv(y, t, beta, mu, u, tao, alpha1, alpha2):
    S, I = y
    dSdt = -beta * S * I + u * (t > tao) * alpha1 * (-S)
    dIdt = beta * S * I - mu * I + u * (t > tao) * alpha2 * (-I)
    return dSdt, dIdt

#add u = 0.5, get the corresponding tao, generate the SI data
# learn u (self.u), the corresponding tao (self.tao)

# Initial conditions vector
y0 = S0, I0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(beta, mu, u, tao, alpha1, alpha2))
S, I = ret.T

# Plot the data on two separate curves for S(t), I(t)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
ax.plot(tSI_vaccination_data[0], tSI_vaccination_data[1], 'navy', alpha=0.9, lw=2, label='Actual Susceptible')
ax.plot(t, S, 'violet', alpha=0.5, lw=2, label='Learnable Param Susceptible', linestyle='dashed')
ax.plot(tSI_vaccination_data[0], tSI_vaccination_data[2], 'dodgerblue', alpha=0.9, lw=2, label='Actual Infected')
ax.plot(t, I, 'darkgreen', alpha=0.5, lw=2, label='Learnable Param Infected', linestyle='dashed')
ax.set_xlabel('Time /days')
#ax.set_ylabel('Number (1000s)')
#ax.set_ylim(0,1.2)
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()

In [ ]:
#calculate relative MSE loss
import math

S_total_loss = 0
S_den = 0
I_total_loss = 0
I_den = 0
for timestep in range(len(t)):
  S_value = tSI_vaccination_data[1][timestep] - S[timestep]
  S_total_loss += S_value**2
  S_den += (tSI_vaccination_data[1][timestep])**2
  I_value = tSI_vaccination_data[2][timestep] - I[timestep]
  I_total_loss += I_value**2
  I_den += (tSI_vaccination_data[2][timestep])**2

S_total_loss = math.sqrt(S_total_loss/S_den)

I_total_loss = math.sqrt(I_total_loss/I_den)
print('S_total_loss: ', S_total_loss)
print('I_total_loss: ', I_total_loss)